In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation

In [2]:
imgw = 64
imgh = 64
nb_train_samples = 8000
nb_test_samples = 2000
epochs = 25
batch_size = 40
input_shape = (64,64,3)

DNN_model_path = 'model/dnn_model.h5'
CNN_model_path = 'model/cnn_model.h5'
TARGET_IMAGE = 'dataset\Tungro\TUNGRO1_004.jpg'

In [3]:
main_path = 'dataset'
generator = ImageDataGenerator(rescale=1./255,
                               validation_split=0.2)

train_datagen = generator.flow_from_directory(main_path,
                                              target_size=(imgw,imgh),
                                              class_mode='categorical',
                                              batch_size=32,
                                              subset='training')

valid_datagen = generator.flow_from_directory(main_path,
                                              target_size=(imgw,imgh),
                                              class_mode='categorical',
                                              batch_size=16,
                                              subset='validation')

Found 4747 images belonging to 4 classes.
Found 1185 images belonging to 4 classes.


In [4]:
for data_batch, labels_batch in train_datagen:
    print('Data batch shape:', data_batch.shape)
    print('Labels batch shape:', labels_batch.shape)
    break

Data batch shape: (32, 64, 64, 3)
Labels batch shape: (32, 4)


# DNN Model

In [5]:
def image_to_array(path):
    img = load_img(path)
    x = img_to_array(img)
    x = x.reshape((1,)+x.shape)
    return x

In [6]:
dnn_model = Sequential()

dnn_model.add(Flatten(input_shape = input_shape))
dnn_model.add(Dense(128))
dnn_model.add(Activation('relu'))
dnn_model.add(Dropout(0.5))
dnn_model.add(Dense(64))
dnn_model.add(Activation('relu'))
dnn_model.add(Dropout(0.5))
dnn_model.add(Dense(1))
dnn_model.add(Activation('tanh'))
dnn_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 128)               1572992   
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 activation_1 (Activation)   (None, 64)                0         
                                                                 
 dropout_1 (Dropout)         (None, 64)                

In [7]:
dnn_model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [8]:
dnn_model.fit_generator(
        train_datagen,
        epochs = epochs,
        validation_data = valid_datagen
        )

Epoch 1/25


C:\Users\wilda\AppData\Local\Temp\ipykernel_30636\4054633166.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  dnn_model.fit_generator(




149/149 [==============================] - 35s 232ms/step - loss: 3.8944 - accuracy: 0.7413 - val_loss: 3.8562 - val_accuracy: 0.7500
Epoch 2/25
149/149 [==============================] - 6s 43ms/step - loss: 3.8602 - accuracy: 0.7489 - val_loss: 3.8562 - val_accuracy: 0.7500
Epoch 3/25
149/149 [==============================] - 6s 42ms/step - loss: 3.8589 - accuracy: 0.7491 - val_loss: 3.8562 - val_accuracy: 0.7500
Epoch 4/25
149/149 [==============================] - 6s 42ms/step - loss: 3.8589 - accuracy: 0.7496 - val_loss: 3.8562 - val_accuracy: 0.7500
Epoch 5/25
149/149 [==============================] - 6s 42ms/step - loss: 3.8572 - accuracy: 0.7498 - val_loss: 3.8562 - val_accuracy: 0.7500
Epoch 6/25
149/149 [==============================] - 7s 44ms/step - loss: 3.8571 - accuracy: 0.7499 - val_loss: 3.8562 - val_accuracy: 0.7500
Epoch 7/25
149/149 [==============================] - 6s 43ms/step - loss: 3.8594 - accuracy: 0.7498 - val_loss: 3.8562 - val_accuracy: 0.7500
Epoch 

In [9]:
dnn_model.save_weights(DNN_model_path)

In [10]:
dnn_model.evaluate(train_datagen)

149/149 [==============================] - 4s 29ms/step - loss: 3.8562 - accuracy: 0.7500


[3.856235980987549, 0.75]

In [11]:
# def predict_model(model_path = DNN_model_path, img_path = TARGET_IMAGE):
#     img = load_img(img_path, target_size = (imgw, imgh))
#     img = img_to_array(img)/255
#     img = img.reshape((1,) + img.shape)
    
#     model = dnn_model((64,64,3))
#     model.load_weights(DNN_model_path)
    
#     print(model.predict(img))

In [12]:
# predict_model()

# CNN Model

In [13]:
from tensorflow.keras.layers import Dropout, Activation, Flatten, Dense, Conv2D, MaxPooling2D

In [14]:
# Set up the model using TensorFlow/Keras
cnn_model = Sequential()
cnn_model.add(Conv2D(96, kernel_size=11, strides=4, activation='relu', input_shape=input_shape))
cnn_model.add(MaxPooling2D(pool_size=3, strides=2))
cnn_model.add(Conv2D(256, kernel_size=5, padding='same', activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=3, strides=2))
cnn_model.add(Conv2D(384, kernel_size=3, padding='same', activation='relu'))
cnn_model.add(Conv2D(384, kernel_size=3, padding='same', activation='relu'))
cnn_model.add(Conv2D(256, kernel_size=3, padding='same', activation='relu'))
cnn_model.add(MaxPooling2D((3,2), padding='same'))
cnn_model.add(Flatten())
cnn_model.add(Dense(4096, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(4096, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(1000, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(4, activation="softmax"))
cnn_model.summary()

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 6, 6, 96)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 256)         614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 2, 2, 256)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 384)         885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 384)       

In [15]:
# class myCallback(tf.keras.callbacks.Callback):
    
#     def on_epoch_end(self, epoch, logs={}):
#         if logs.get('val_accuracy') > 0.99:
#             self.model.stop_training = True
            
class myCallback(tf.keras.callbacks.Callback):
    def __init__(self, patience=10, delta=0.001, max_acc = 0.99):
        super(myCallback, self).__init__()
        self.patience = patience
        self.delta = delta
        self.wait = 0
        self.best_val_acc = -float('inf')
        self.max_acc = max_acc

    def on_epoch_end(self, epoch, logs={}):
        current_val_acc = logs.get('val_accuracy')
        if current_val_acc is None:
            return

        if current_val_acc > self.best_val_acc + self.delta:
            self.best_val_acc = current_val_acc
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
                print("\nTraining stopped as val_accuracy did not improve for {} epochs.".format(self.patience))
            
        if logs.get('val_accuracy') >=  self.max_acc and logs.get('accuracy') >=  self.max_acc:
            self.model.stop_training = True
            

In [ ]:
callback = myCallback()
cnn_history = cnn_model.fit_generator(train_datagen,
          epochs=10,
          validation_data=valid_datagen,
          callbacks=callback)
cnn_history.save_weights(CNN_model_path)

In [ ]:
cnn_model.evaluate(train_datagen)

149/149 [==============================] - 9s 63ms/step - loss: 1.3831 - accuracy: 0.2696


[1.383091926574707, 0.26964399218559265]

# History Plotting

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc_loss = [['accuracy', 'val_accuracy'],
            ['loss', 'val_loss']]

for i in acc_loss:
    
    plt.plot(dnn_model.history[i[0]], 'g-o')
    plt.plot(dnn_model.history[i[1]], 'b-+')
    plt.title(f'model {i[0]} and {i[1]}')
    plt.ylabel(f'{i[0]}')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()
    